In [12]:
# LIBRARIES


import gradio as gr
import pandas as pd
import requests




In [13]:
# ===== In-memory storage for CRUD =====
weather_db = {}  # key = location, value = dict with weather info

# ===== Function to fetch weather data =====
def fetch_weather(city):
    try:
        if not city.strip():
            return None, "❌ Please enter a valid city name."

        geocode_url = f"https://nominatim.openstreetmap.org/search?city={city}&format=json"
        headers = {"User-Agent": "HuggingFaceSpacesWeatherApp/1.0 (your_email@example.com)"}
        geocode_resp = requests.get(geocode_url, headers=headers).json()

        if not geocode_resp:
            return None, f"❌ Could not find location: {city}"

        lat = geocode_resp[0]["lat"]
        lon = geocode_resp[0]["lon"]

        weather_url = (
            f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}"
            f"&daily=temperature_2m_max,temperature_2m_min,weathercode&timezone=auto"
        )
        weather_resp = requests.get(weather_url).json()

        if "daily" not in weather_resp:
            return None, "⚠️ Weather data unavailable for this location."

        dates = weather_resp["daily"]["time"]
        max_temps = weather_resp["daily"]["temperature_2m_max"]
        min_temps = weather_resp["daily"]["temperature_2m_min"]
        weather_codes = weather_resp["daily"]["weathercode"]

        weather_dict = {
            0: "☀️ Clear sky",
            1: "🌤️ Mainly clear",
            2: "🌦️ Partly cloudy",
            3: "☁️ Overcast",
            45: "🌫️ Fog",
            48: "🌫️ Rime fog",
            51: "🌦️ Drizzle",
            61: "🌧️ Rain",
            71: "❄️ Snow",
            95: "⛈️ Thunderstorm"
        }
        descriptions = [weather_dict.get(code, "🌍 Unknown") for code in weather_codes]

        df = pd.DataFrame({
            "Date": dates,
            "Min Temp (°C)": min_temps,
            "Max Temp (°C)": max_temps,
            "Condition": descriptions
        })

        summary = f"✅ Weather data for **{city.title()}** (7-Day Forecast):\n\n"
        for i in range(len(dates)):
            summary += f"{dates[i]}: {min_temps[i]}°C - {max_temps[i]}°C | {descriptions[i]}\n"

        return {"summary": summary, "dataframe": df}, None

    except Exception as e:
        return None, f"⚠️ Error: {str(e)}"

In [14]:
# ===== CRUD Operations =====
def create_location(city):
    weather, error = fetch_weather(city)
    if error:
        return str(error), list(weather_db.keys())
    weather_db[city] = weather
    return f"✅ Added {city} to the list.", list(weather_db.keys())

def read_locations():
    if not weather_db:
        return "No locations saved yet.", []
    return "Saved Locations:", list(weather_db.keys())

def update_location(old_city, new_city):
    if old_city not in weather_db:
        return f"❌ {old_city} not found.", list(weather_db.keys())
    weather, error = fetch_weather(new_city)
    if error:
        return str(error), list(weather_db.keys())
    del weather_db[old_city]
    weather_db[new_city] = weather
    return f"✅ Updated {old_city} to {new_city}.", list(weather_db.keys())

def delete_location(city):
    if city not in weather_db:
        return f"❌ {city} not found.", list(weather_db.keys())
    del weather_db[city]
    return f"✅ Deleted {city}.", list(weather_db.keys())


In [15]:
# Weather fetch for any input
def get_weather_summary(city):
    weather, error = fetch_weather(city)
    if error:
        return error, None
    df = weather["dataframe"]
    csv_path = f"{city}_weather.csv"
    df.to_csv(csv_path, index=False)
    return weather["summary"], csv_path


In [16]:
# ===== PM Accelerator Info =====
def pma_info():
    return (
        "💡 **About Product Manager Accelerator (PMA)**\n\n"
        "Product Manager Accelerator (PMA) empowers aspiring and current product professionals "
        "to master product management through mentorship, real-world projects, and leadership training. "
        "Our mission is to build future-ready PMs who drive innovation and impact in their industries.\n\n"
        "🔗 Visit us on [LinkedIn](https://www.linkedin.com/company/product-manager-accelerator/)"
    )




In [17]:
# ===== Custom CSS =====
custom_css = """
#header {background: linear-gradient(90deg, #1E90FF, #00CED1); color: white; border-radius: 15px; padding: 15px; text-align:center;}
.section {background-color: #F0F8FF; padding: 15px; border-radius: 15px; margin-bottom: 10px;}
.button-primary {background: linear-gradient(90deg, #FF7F50, #FF6347); color: white; border-radius: 10px; padding: 10px 15px; font-weight:bold;}
.button-secondary {background: linear-gradient(90deg, #32CD32, #228B22); color: white; border-radius: 10px; padding: 10px 15px; font-weight:bold;}
textarea {border-radius: 10px; border: 2px solid #1E90FF; padding:5px;}
"""

In [18]:

# ===== Gradio UI =====
with gr.Blocks(css=custom_css) as demo:
    # Header
    gr.Markdown("## 🌦️ MyWeatherApp", elem_id="header")
    gr.Markdown("### Developed by **Sareena** | Powered by **PMA**")

    # PMA Info
    with gr.Row(elem_classes="section"):
        info_btn = gr.Button("ℹ️ PMA Info", elem_classes="button-secondary")
        pma_output = gr.Markdown()
    info_btn.click(fn=pma_info, inputs=None, outputs=pma_output)

    # Location input and Check Weather button
    with gr.Row(elem_classes="section"):
        city_input = gr.Textbox(label="Location", placeholder="Enter city/ZIP/landmark")
    get_weather_btn = gr.Button("🔍 Check Weather", elem_classes="button-primary")
    weather_output = gr.Textbox(label="Weather Summary", lines=12, interactive=False)

    get_weather_btn.click(get_weather_summary, inputs=city_input, outputs=[weather_output, gr.File()])

    # CRUD section
    new_city_input = gr.Textbox(label="New Location (for Update)", placeholder="Optional")
    with gr.Row(elem_classes="section"):
        add_btn = gr.Button("➕ Add Location", elem_classes="button-primary")
        update_btn = gr.Button("✏️ Update Location", elem_classes="button-primary")
        delete_btn = gr.Button("🗑️ Delete Location", elem_classes="button-primary")
        list_btn = gr.Button("📋 List Locations", elem_classes="button-primary")
    location_output = gr.Textbox(label="Saved Locations", lines=5, interactive=False)

    add_btn.click(create_location, inputs=city_input, outputs=[location_output, location_output])
    update_btn.click(update_location, inputs=[city_input, new_city_input], outputs=[location_output, location_output])
    delete_btn.click(delete_location, inputs=city_input, outputs=[location_output, location_output])
    list_btn.click(read_locations, inputs=None, outputs=[location_output, location_output])

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c9d7d133fbfb517b9c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
